## In this jupyter notebook, we are going to augment our blackjack table dataset so that we have more images. 

### Goal: Get 400 Blackjack Table images Then, get 600 other background images (non-blackjack).  Run chip placing algorithm + truth tables, push to the github. 


### Below are the properties of an image we can change:


- Hue (color shift, measured by angle on the color wheel)
- Saturation (% purity of the color vs. gray)
- Value (brightness/intensity of color, like a total brightness)
- Contrast (difference between brightest and darkest parts of an image, like a relative brightness) 
- Quantization (# of colors, or color complexity, in the image)

The different ways that we can adjust these are as follows:

- OpenCV: converting an image to HSV color space lets us adjust Hue, Saturation, and Value easily, and we can adjust Qshift with k-means clustering. Contrast is a little harder but doable. 
- Pytorch Colorjitter: one-line adjustment for hue, saturation, value, and contrast. No easy adjustment for Qshift

Other solutions weren't nearly as good as OpenCV or Pytorch. 

Because of this, let's use OpenCV instead of Pytorch Colorjitter.

Things we do NOT want to shift:

1. Rotation angle.
2. Random croppings
3. Other qualtiies that would make the image not resemble a blackjack table.

In [44]:
import cv2
import numpy as np
import os
from PIL import Image
from torchvision import transforms

In [83]:
def adjust_hsv(image, hue_shift=0, saturation_scale=1.0, value_scale=1.0):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image[:, :, 0] = (hsv_image[:, :, 0] + hue_shift) % 180
    hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * saturation_scale, 0, 255)
    hsv_image[:, :, 2] = np.clip(hsv_image[:, :, 2] * value_scale, 0, 255)
    return cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

def apply_quantization(image, k=8):
    Z = image.reshape((-1, 3))
    Z = np.float32(Z)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, center = cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    res = center[labels.flatten()]
    return res.reshape((image.shape))

def adjust_contrast(image, factor=1.0):
    # Adjust contrast; factor > 1.0 increases contrast, factor < 1.0 decreases contrast
    pil_img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    enhancer = ImageEnhance.Contrast(pil_img)
    img_contrasted = enhancer.enhance(factor)
    return cv2.cvtColor(np.array(img_contrasted), cv2.COLOR_RGB2BGR)

def process_chips(chips_dir, output_dir, aug_settings):
    for root, dirs, files in os.walk(chips_dir):
        for file in tqdm(files):
            if file.endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file)

                for i in range(aug_settings["image_copies"]):
                    # For each simple image, make image_copy copies, e.g. for the best we make 30
                    image = cv2.imread(file_path)
                    
                    # Apply HSV adjustments if enabled
                    if aug_settings.get("hue"):
                        image = adjust_hsv(image, hue_shift=np.random.uniform(*aug_settings["hue_vals"]))
                    if aug_settings.get("saturation"):
                        image = adjust_hsv(image, saturation_scale=np.random.uniform(*aug_settings["sat_vals"]))
                    if aug_settings.get("value/brightness"):
                        image = adjust_hsv(image, value_scale=np.random.uniform(*aug_settings["value_vals"]))
                    
                    # Apply contrast adjustment if enabled
                    if aug_settings.get("contrast"):
                        image = adjust_contrast(image, factor=np.random.uniform(*aug_settings["contrast_vals"]))
    
                    # Apply quantization if enabled, selecting a random k within the specified range
                    if aug_settings.get("quantization"):
                        k = random.randint(*aug_settings["quantization_vals"])
                        image = apply_quantization(image, k=k)
    
                    os.makedirs(output_dir, exist_ok=True)
                    save_path = os.path.join(output_dir, file[:-4] + str(i) + aug_settings["extra_file_name"])
                    cv2.imwrite(save_path, image)

In [88]:
aug_settings = {
    "hue": True, 
    "hue_vals": (-7, 7), 
    "saturation": True, 
    "sat_vals": (0.3, 1.7),
    "value/brightness": True,
    "value_vals": (0.6, 1.4),
    "contrast": True,
    "contrast_vals": (0.5, 1.5),
    "quantization": True,
    "quantization_vals": (5, 12),
    "image_copies": 2, 
    "extra_file_name": "2.jpg"
}

chips_dir = '../Backgrounds/Blackjack_Esque_Environment'
output_dir = '../Backgrounds/Aug_Blackjack2/Other'
process_chips(chips_dir, output_dir, aug_settings)
print("Done")

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:13<00:00,  1.68it/s]

Done


In [30]:
extra_val = 0